In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from OGMM import OGMM, init_cov_and_T_split
from sklearn.mixture import GaussianMixture
from scipy.special import logsumexp

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
MGR_data = np.load(r'C:\Users\SL276123\Documents\Online DaDiL\MGR_WBTransport\MGR_WBTransport\data\MGR100.npy')
MGR_features = MGR_data[:, :-2]
MGR_labels = MGR_data[:, -2]
MGR_domains = MGR_data[:, -1]
X_train, X_test, y_train, y_test = train_test_split(MGR_features, MGR_labels, test_size=0.2, random_state=0)
X_train_t = torch.from_numpy(X_train)
X_test_t = torch.from_numpy(X_test)

In [3]:
CSTR_data = np.load('./data/cstr_acfmeandata.npy')
CSTR_features = CSTR_data[:, :-4]
CSTR_labels = CSTR_data[:, -4]
CSTR_domains = CSTR_data[:, -3]
X_train, X_test, y_train, y_test = train_test_split(CSTR_features, CSTR_labels, test_size=0.2, random_state=0)
X_train_t = torch.from_numpy(X_train)
X_test_t = torch.from_numpy(X_test)
X_train_target, X_test_target, y_train_target, y_test_target = train_test_split(CSTR_features[np.where(CSTR_domains==6)[0], :], CSTR_labels[np.where(CSTR_domains==6)[0]], test_size=0.2, random_state=0)
X_train_target_t = torch.from_numpy(X_train_target)
X_test_target_t = torch.from_numpy(X_test_target)

## Log likelihood function

In [4]:
GMM_sklearn = GaussianMixture(n_components=26, random_state=0).fit(X_train_target)

In [5]:
GMM_sklearn.sample(100)

<class 'int'>


(array([[0.01870513, 0.03050341, 0.71569505, ..., 0.02910691, 0.02180226,
         0.22188449],
        [0.03656846, 0.0288222 , 0.74874293, ..., 0.02690639, 0.02083822,
         0.22306459],
        [0.03367389, 0.02711883, 0.69727668, ..., 0.02813525, 0.01795489,
         0.19968224],
        ...,
        [0.01859864, 0.02835531, 0.01726093, ..., 0.02747718, 0.01936281,
         0.12701677],
        [0.70281891, 0.02968151, 0.03049145, ..., 0.03018488, 0.00279911,
         0.18282624],
        [0.70356649, 0.02851049, 0.02312163, ..., 0.02928326, 0.0154037 ,
         0.19380079]]),
 array([ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  3,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,
         6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,
         8,  8,  9,  9,  9, 10, 10, 11, 12, 12, 12, 13, 13, 14, 15, 16, 17,
        17, 17, 17, 17, 17, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21,
        21, 21, 21, 21, 21, 2

In [85]:
GMM_sklearn.weights_

array([0.06730769, 0.02403846, 0.11538783, 0.01442308, 0.04326919,
       0.0625    , 0.06730769, 0.03365385, 0.03846154, 0.03846158,
       0.00961538, 0.01442308, 0.02884615, 0.00961538, 0.04326923,
       0.03846154, 0.05288462, 0.03846154, 0.00480769, 0.03365385,
       0.01442308, 0.10095832, 0.03365385, 0.04807692, 0.01923077,
       0.00480769])

In [76]:
np.sum(GMM_sklearn.score_samples(X_test_target))

-3048.4572248600402

In [77]:
def normal_pdf(x, mean, cov, bib='numpy'):
    if bib == 'numpy':
        return (2*np.pi)**(-len(x)/2) * np.linalg.det(cov)**(-1/2) * np.exp(-1/2 * (x - mean).T @ np.linalg.inv(cov) @ (x - mean))
    else:
        return (2*torch.pi)**(-len(x)/2) * torch.linalg.det(cov)**(-1/2) * torch.exp(-1/2 * (x - mean).T @ torch.inverse(cov) @ (x - mean))

def log_normal_pdf(x, mean, cov, bib='numpy'):
    if bib == 'numpy':
        return -len(x)/2*np.log(2*np.pi) - 1/2*(np.log(np.linalg.det(cov)) + (x - mean).T @ np.linalg.inv(cov) @ (x - mean))
    else:
        return -len(x)/2*torch.log(torch.tensor(2*torch.pi)) - 1/2*(torch.log(torch.linalg.det(cov)) + (x - mean).T @ torch.inverse(cov) @ (x - mean))

def log_likelihood(mixture, X, bib='numpy'):
    log_pdf = np.zeros((X.shape[0], len(mixture)))
    for i in range(X.shape[0]):
        for j in range(len(mixture)):
            log_pdf[i, j] = np.log(mixture[j][0]) + log_normal_pdf(X[i], mixture[j][2], mixture[j][3], bib)
            
    return np.sum(logsumexp(log_pdf, axis=1))

In [78]:
means = GMM_sklearn.means_
weights = GMM_sklearn.weights_
cov = GMM_sklearn.covariances_
GMM_sklearn_list = [[weights[i], 1, means[i], cov[i]] for i in range(len(weights))]

In [80]:
ll_sklearn = log_likelihood(GMM_sklearn_list, X_test_target)
ll_sklearn

-3048.4572248601307

In [81]:
range_data = np.mean(np.max(X_train_target, axis=0) - np.min(X_train_target, axis=0))

K_max = 26
cov_init, T_split = init_cov_and_T_split(K_max, range_data, X_train_target.shape[1])
GMM_online = OGMM(X_train_target_t, K_max, cov_init, 0.1, T_split)

In [82]:
GMM_online

[[tensor(0.2712),
  tensor(55.3246),
  tensor([0.0232, 0.0283, 0.0169, 0.0282, 0.0130, 0.1923, 0.0133, 0.1826, 0.0189,
          0.2090, 0.0263, 0.0280, 0.0166, 0.1617], dtype=torch.float64),
  tensor([[ 1.5242e-04, -3.6752e-06,  1.5866e-06,  9.3047e-08, -1.4057e-05,
           -4.7022e-04, -1.8196e-05, -4.4084e-04, -5.4470e-05, -5.8307e-04,
            9.0108e-05, -1.1754e-05, -1.8171e-05, -4.3822e-04],
          [-3.6752e-06,  2.4137e-06,  1.2632e-07, -3.3316e-07,  6.8625e-07,
            3.3961e-05,  1.0787e-06,  3.2836e-05,  3.5717e-06,  3.9173e-05,
           -4.3723e-07,  6.7066e-07, -1.5462e-06,  3.2068e-05],
          [ 1.5866e-06,  1.2632e-07,  2.7999e-05,  5.4704e-08,  1.4058e-06,
            1.1546e-04,  4.6032e-06,  1.0731e-04, -1.0493e-05,  1.6589e-04,
           -4.2179e-06, -4.4356e-07,  1.3497e-05,  9.7715e-05],
          [ 9.3047e-08, -3.3316e-07,  5.4704e-08,  1.4355e-06, -2.6144e-08,
           -5.5352e-06,  5.0659e-07, -5.8282e-06, -1.3874e-06, -8.1039e-06,
        

In [83]:
ll_online = log_likelihood(GMM_online, X_test_target_t, 'torch')
ll_online

2055.9551331631137

In [84]:
(ll_online - ll_sklearn)/ll_sklearn

-1.6744247931041398

## PCA

In [12]:
def approximate_data(data_features, n_neighbors, n_components):
    NN = NearestNeighbors(n_neighbors=n_neighbors)
    NN.fit(data_features)

    _, indices = NN.kneighbors(data_features)
    NN_features = data_features[indices]


    pca = PCA(n_components=n_components)
    pca.whiten = False
    data_approx = np.zeros(data_features.shape)

    for i in range(NN_features.shape[0]):
        X_trans = pca.fit_transform(NN_features[i, :, :])
        X_app = pca.inverse_transform(X_trans)[0]
        data_approx[i] = X_app.reshape(-1,)
    
    return data_approx, pca.explained_variance_ratio_

In [17]:
def approx_classification_error(data_features, data_labels, data_approx):
    #classifier = RandomForestClassifier(criterion = 'log_loss')
    classifier_raw = SVC()
    classifier_approx = SVC()

    X_train, X_test, y_train, y_test = train_test_split(data_features, data_labels, test_size=0.2, random_state=0)
    X_train_approx, X_test_approx, y_train_approx, y_test_approx = train_test_split(data_approx, data_labels, test_size=0.2, random_state=0)

    classifier_raw.fit(X_train, y_train)
    classifier_approx.fit(X_train_approx, y_train_approx)

    approx_error_list = []
    for i in range(1):
        y_pred = classifier_raw.predict(X_test)
        y_pred_approx = classifier_approx.predict(X_test_approx)

        accuracy_raw = accuracy_score(y_test, y_pred)
        accuracy_approx = accuracy_score(y_test_approx, y_pred_approx)
        print(accuracy_raw, accuracy_approx)
        approx_error_list.append(np.abs((accuracy_approx - accuracy_raw))/accuracy_raw)

    return np.mean(approx_error_list)

PCA MGR

In [18]:
target_domain = 4
target_features = MGR_features[np.where(MGR_domains==target_domain)[0], :]
target_labels = MGR_labels[np.where(MGR_domains==target_domain)[0]]
target_domains = MGR_domains[np.where(MGR_domains==target_domain)[0]]

sources_features = MGR_features[np.where(MGR_domains==target_domain)[0], :]
sources_labels = MGR_labels[np.where(MGR_domains==target_domain)[0]]
sources_domains = MGR_domains[np.where(MGR_domains==target_domain)[0]]

In [19]:
data_approx, ex_var = approximate_data(MGR_features, 10, 1)
print(ex_var)

approx_classification_error(MGR_features, MGR_labels, data_approx)

[0.80543947]
0.247 0.245


0.008097165991902841

PCA CSTR

In [20]:
target_domain = 6
target_features = CSTR_features[np.where(CSTR_domains==target_domain)[0], :]
target_labels = CSTR_labels[np.where(CSTR_domains==target_domain)[0]]
target_domains = CSTR_domains[np.where(CSTR_domains==target_domain)[0]]

sources_features = CSTR_features[np.where(CSTR_domains!=target_domain)[0], :]
sources_labels = CSTR_labels[np.where(CSTR_domains!=target_domain)[0]]
sources_domains = CSTR_domains[np.where(CSTR_domains!=target_domain)[0]]


In [21]:
data_approx, ex_var = approximate_data(CSTR_features, 10, 1)
print(ex_var)

approx_classification_error(CSTR_features, CSTR_labels, data_approx)

[0.31433579]
0.9230769230769231 0.9195804195804196


0.00378787878787884

In [16]:
cstr_sources = np.concatenate([sources_features, sources_labels.reshape(-1, 1), sources_domains.reshape(-1, 1)], axis=1)

cstr_features_target_approx = approximate_data(target_features, 10, 1)
cstr_target_approx = np.concatenate([cstr_features_target_approx[0], target_labels.reshape(-1, 1), target_domains.reshape(-1, 1)], axis=1)

cstr_with_target_approx = np.concatenate([cstr_sources, cstr_target_approx], axis=0)

np.save('../CSTR/data/cstr_with_target_approx1.npy', cstr_with_target_approx)

Incremental PCA

In [28]:
n_components = 5
ipca = IncrementalPCA(n_components=n_components)
ipca.whiten = False

target_features_iapprox = np.zeros(target_features.shape)
target_features_iapprox[:n_components-1, :] = target_features[:n_components-1, :]
for i in range(n_components, target_features.shape[0]):
    X_batch = target_features[i-n_components:i, :]
    ipca.partial_fit(X_batch)
    approx_x = ipca.inverse_transform(ipca.transform(X_batch[[-1], :]))
    target_features_iapprox[i-1, :] = approx_x

cstr_target_iapprox = np.concatenate([target_features_iapprox, target_labels.reshape(-1, 1), target_domains.reshape(-1, 1)], axis=1)
cstr_with_target_iapprox = np.concatenate([cstr_sources, cstr_target_iapprox], axis=0)

In [29]:
np.save('../CSTR/data/cstr_with_target_iapprox5.npy', cstr_with_target_iapprox)

In [30]:
approx_classification_error(CSTR_features, CSTR_labels, cstr_with_target_iapprox[:, :-2])

0.9230769230769231 0.9248251748251748


0.0018939393939392997

Online PCA, batch size = 1

In [ ]:
# Specify the number of components for PCA
n_components = 3

# Create a PCA object
pca = PCA(n_components=n_components)
pca.whiten = False

# Initialize a running mean for each component
mean_running = np.zeros(n_components)

target_features_online_approx = np.zeros(target_features.shape)
# Perform online learning with batch size 1
for i, x in enumerate(target_features):
    # Update running mean
    mean_running = (i * mean_running + x) / (i + 1)

    # Update covariance matrix (assuming zero mean)
    cov_running = np.outer(x - mean_running, x - mean_running)

    # Update PCA components
    pca.components_ = (i * pca.components_ + cov_running) / (i + 1)

    x_approx = pca.inverse_transform(pca.transform(x))
    target_features_online_approx[i, :] = x_approx

cstr_target_online_approx = np.concatenate([target_features_online_approx, target_labels.reshape(-1, 1), target_domains.reshape(-1, 1)], axis=1)
cstr_with_target_online_approx = np.concatenate([cstr_sources, cstr_target_online_approx], axis=0)

approx_classification_error(CSTR_features, CSTR_labels, cstr_with_target_online_approx[:, :-2])